In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import cv2
from gait import *
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from angle_keypose import angle_ims

In [13]:
# 1,1,3,128,128 => 1-sample/batch_size, 1-channels, 3 - #frames, 128 - height, 128 - width
x = torch.rand((8,1,3,120,160))
y = torch.rand((1,1,1,64,64))

In [24]:
class Conv3D(nn.Module):

    '''
    Performs 3D convolution
    '''

    def __init__(self):

        super(Conv3D, self).__init__()
        self.conv1 = self._convblock(1,16,3,3,False)
        self.conv2 = self._convblock(16,32,1,5,False)
        self.conv3 = self._convblock(32,16,1,7,False)
        self.conv4 = self._convblock(16,1,1,9,True)

    def _convblock(self,in_channels,out_channels,ksized,ksize,last):
        '''
        Makes a block of layers (Conv3d,ReLU,Maxpool3d,BatchNorm3d(only if !last))
        '''

        l1 = nn.Conv3d(in_channels=in_channels,out_channels=out_channels,kernel_size=(ksized,ksize,ksize))
        l2 = nn.ReLU()
        
        if last:
            return nn.Sequential(l1,l2)
        else:
            l3 = nn.MaxPool3d((1, 1, 1))
            l4 = nn.BatchNorm3d(out_channels)
            return nn.Sequential(l1,l2,l3,l4)
        
        
    def forward(self,x):

        out = self.conv1(x)
        # out = self.conv2(out)
        # out = self.conv3(out)
        out = self.conv4(out)

        return out


In [25]:
model = Conv3D()

In [26]:
out = model(x)
out.shape
# out.reshape((8,1,112,152))

torch.Size([8, 1, 1, 110, 150])

In [23]:
out.detach().numpy().reshape(8,-1).shape

(8, 18096)

In [31]:
test = np.random.randn(8,1,3,120,160)
test.shape

(8, 1, 3, 120, 160)

In [33]:
x = torch.rand((8,1,3,120,160))

In [6]:
class PEIData(Dataset):

    def __init__(self, angle=0, keypose = 4, data_path='/home/ishikaa/Downloads',transform=None):
        """
        Custom dataset for images of a certain keypose at a given angle.

        Args:
            angle (int)     : Angle
            keypose (int)   : Key-pose/Cluster
            data_path (str) : Path where dataset is downloaded
        """
        ds = angle_ims(exp=1,angle=0,keypose = 4,data_path='/home/ishikaa/Downloads')

        images = np.empty((len(ds),3,ds[0][0].shape[0]//2,ds[0][0].shape[1]//2))
        for i in range(len(ds)):
            images[i] = np.asarray([preprocess(im)/255 for im in ds[i]])
        images = images.reshape(images.shape[0],1,images.shape[1],images.shape[2],images.shape[3])

        #Avg PEI after PCA .
        self.y = 

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        single_image_label = self.labels[idx]
        single_image = self.images[idx]

SyntaxError: invalid syntax (<ipython-input-6-9b301b8775dc>, line 20)

In [5]:
ds = angle_ims(exp=1,angle=0,keypose = 4,data_path='/home/ishikaa/Downloads')

In [6]:
images = np.empty((len(ds),3,ds[0][0].shape[0]//2,ds[0][0].shape[1]//2))
for i in range(len(ds)):
    images[i] = np.asarray([preprocess(im)/255 for im in ds[i]])
# images = images.reshape(images.shape[0],1,images.shape[1],images.shape[2],images.shape[3])

In [ ]:
labels = gait.fetch_labels(label_angle=angle,save=False,override=True)